# <font color='#4C5FDA'>**Breast Cancer Detection Based on CNNs Using Thermal Imaging** </font>

Original paper by Juan Pablo Zuluaga, Zeina Al Masry, Khaled Benaggoune, Safa Meraghni & Noureddine Zerhouni: [A CNN-based methodology for breast cancer diagnosis using thermal images](https://www.tandfonline.com/doi/full/10.1080/21681163.2020.1824685)

In [172]:
#@title **Importamos librerías necesarias**

# Pytorch essentials
import torch
import torch.nn as nn

**The Xception architecture**: the data first goes through the entry flow, then through the middle flow which is repeated eight times, and finally through the exit flow. Note that all Convolution and SeparableConvolution layers are followed by batch normalization [7] (not included in the diagram). All SeparableConvolution layers use a depth multiplier of 1 (no depth expansion)

In this architecture we first perform the 1x1 convolution and then the 3x3 separable convolution.

<div align="center"> <image src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*J8dborzVBRBupJfvR7YhuA.png" width=600>  </div>

La arquitectura completa tiene este aspecto:

<div align="center"> <image src="https://maelfabien.github.io/assets/images/xception.jpg" width=800>  </div>



### <font color='#52F17F'>**Model from scratch**</font>

#### <font color='#8203b1'>**Entry flow**</font>

In [173]:
class DoubleConvBlock(nn.Module):
  def __init__(self, in_channels, out_channels):
     super().__init__()
    # Doble bloque convolucional al inicio de Xception
     self.double_conv = nn.Sequential(
         nn.Conv2d(in_channels, out_channels//2, kernel_size=3, stride=2, bias=False),
         nn.BatchNorm2d(out_channels//2),
         nn.ReLU(inplace=True),
         nn.Conv2d(out_channels//2, out_channels, kernel_size=3, stride=1, bias=False),
         nn.BatchNorm2d(out_channels),
         nn.ReLU(inplace=True)
     )

  def forward(self, x):
    return self.double_conv(x)
  
class SeparableConv2d(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    self.depth_wise_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, groups=in_channels, padding=1, bias=False)
    self.one_by_one = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)

  def forward(self, x):
    x = self.depth_wise_conv(x)
    x = self.one_by_one(x)
    return x

class EntryFlowModule(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    # first one by one
    self.one_by_one = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2, bias=False)
    self.bn1 = nn.BatchNorm2d(out_channels)

    # double separable conv 2d with maxpool
    self.double_depth_wise_conv = nn.Sequential(
      SeparableConv2d(in_channels, out_channels),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
      SeparableConv2d(out_channels, out_channels),
      nn.BatchNorm2d(out_channels),
      nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    )
  
  def forward(self, x):
    x1 = self.one_by_one(x)
    x1 = self.bn1(x1)
    x2 = self.double_depth_wise_conv(x)
    return x1+x2

#### <font color='#8203b1'>**Middle flow**</font>

In [174]:
class MiddleFlowModule(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    # triple separable conv 2d
    self.triple_depth_wise_conv = nn.Sequential(
      nn.ReLU(inplace=True),
      SeparableConv2d(in_channels, out_channels),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
      SeparableConv2d(out_channels, out_channels),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
      SeparableConv2d(out_channels, out_channels),
      nn.BatchNorm2d(out_channels),
    )
  
  def forward(self, x):
    x = self.triple_depth_wise_conv(x)
    return x

#### <font color='#8203b1'>**Exit flow**</font>

In [175]:
class ExitFlowModule(nn.Module):
  def __init__(self, in_channels, n_classes):
    super().__init__()

    # first one by one
    self.one_by_one = nn.Conv2d(in_channels, 1024, kernel_size=1, stride=2, bias=False)
    self.bn2 = nn.BatchNorm2d(1024)

    self.relu2 = nn.ReLU()

    # double separable conv 2d with maxpool
    self.double_depth_wise_conv = nn.Sequential(
      SeparableConv2d(in_channels, in_channels),
      nn.BatchNorm2d(in_channels),
      nn.ReLU(inplace=True),
      SeparableConv2d(in_channels, 1024),
      nn.BatchNorm2d(1024),
      nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
    )

    self.last_depth_wise_conv = nn.Sequential(
      SeparableConv2d(1024, 1536),
      nn.BatchNorm2d(1536),
      nn.ReLU(inplace=True),
      SeparableConv2d(1536, 2048),
      nn.BatchNorm2d(2048),
      nn.ReLU(inplace=True)
    )

    self.gap = nn.AdaptiveAvgPool2d((1, 1))
    self.last_fc = nn.Linear(2048, n_classes)
  
  def forward(self, x):
    x1 = self.one_by_one(x)
    x1 = self.bn2(x1)
    x = self.relu2(x)
    x2 = self.double_depth_wise_conv(x)
    x = x1+x2
    x = self.last_depth_wise_conv(x)
    x = self.gap(x)
    x = x.view(x.size(0), -1)
    x = self.last_fc(x)
    return x

#### <font color='#8203b1'>**Full model**</font>

In [176]:
class Xception(nn.Module):
  def __init__(self, n_channels, n_classes):
    super().__init__()
    self.inc = DoubleConvBlock(n_channels, 64) 
    self.mod1 = EntryFlowModule(64, 128)
    self.relu = nn.ReLU(inplace=True)
    self.mod2 = EntryFlowModule(128, 256)
    self.mod3 = EntryFlowModule(256, 728)
    self.middle_flow = nn.Sequential()
    for _ in range(8):
      self.middle_flow.append(MiddleFlowModule(728, 728))
    self.exit_flow = ExitFlowModule(728, n_classes)

  def forward(self, x):
    x = self.inc(x)
    x = self.mod1(x)
    x = self.relu(x)
    x = self.mod2(x)
    x = self.relu(x)
    x = self.mod3(x)
    x = self.middle_flow(x)
    x = self.exit_flow(x)
    return x

In [180]:
# Test the model to see if it gives the expected result.

input_image = torch.rand([2, 1, 299, 299])
print(f"Entrada: {input_image.size(), {input_image.dtype}}")
model = Xception(n_channels=1, n_classes=1)
ouput = model(input_image)
print(f"Salida: {ouput.size(), ouput.dtype}")

Entrada: (torch.Size([2, 1, 299, 299]), {torch.float32})
Salida: (torch.Size([2, 1]), torch.float32)


In [181]:
print(model)

Xception(
  (inc): DoubleConvBlock(
    (double_conv): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (mod1): EntryFlowModule(
    (one_by_one): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (double_depth_wise_conv): Sequential(
      (0): SeparableConv2d(
        (depth_wise_conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (one_by_one): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affi